In [1]:
import os, sys, argparse, importlib, time, inspect
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
if hasattr(__builtins__,'__IPYTHON__'):
    print('Notebook')
    from tqdm.notebook import tqdm
else:
    print('Not notebook')
    from tqdm import tqdm
from tensorflow.keras import backend as K
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_probability as tfp

gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('No GPU detected')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model, Model
import spektral
from sklearn.preprocessing import normalize
from spektral.data import DisjointLoader, BatchLoader, SingleLoader
from importlib import reload
import winsound
import dill
import datetime as dt
import from_config.dev.data_load as dl
import wandb
graph_data=dl.graph_data

Notebook
GPU detected
Not notebook


In [2]:
# with tf.device('/cpu:0'): # if on the cpu
batch_size=512
dataset=graph_data(n_data=100000,         transform_path='db_files/muongun/transformers.pkl',\
             db_path= 'db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db', n_neighbors=10, skip=int(1.4e6-1), restart=0)
loader = DisjointLoader(dataset, batch_size=batch_size, epochs=1)

Loading data to memory


In [4]:
tf.executing_eagerly()
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()
tf.executing_eagerly()

False

In [3]:


from spektral.layers import GraphSageConv, MessagePassing
from spektral.layers.pooling.global_pool import GlobalMaxPool, GlobalAvgPool, GlobalSumPool

from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout, multiply
from tensorflow.keras.activations import tanh, sigmoid
from tensorflow.sparse import SparseTensor

eps=1e-5

print('loading model')

d_act=LeakyReLU(alpha=0.15)

def no_norm(x, training):
  return x

class KHist(Model):
    def __init__(self,hist, n_out = 3, n_sigs=2, K=[1,2], agg_method='min', hidden_states=40, glob=True, conv_layers=2, conv_activation='relu', decode_layers=2, decode_activation=1, regularization=None, dropout=0.2, batch_norm=True, forward=True):
        super().__init__()
        self.hist=hist
        self.n_out=n_out
        self.n_sigs=n_sigs
        self.hidden_states=hidden_states
        self.conv_activation=conv_activation
        self.forward=forward
        self.dropout=dropout
        self.glob=glob
        self.Ks=K
        self.agg_method=agg_method
        self.conv_layers=conv_layers
        self.regularize=regularization
        if type(decode_activation)==str:
          self.decode_activation=tf.keras.activations.get(decode_activation)
        else:
          self.decode_activation=d_act
        self.batch_norm=batch_norm
        # Define layers of the model

        self.MPs      = [SGConv(hidden_states, hidden_states, K=K, agg_method=self.agg_method, dropout = dropout) for K in self.Ks]

        self.GCNs    = [GraphSageConv(hidden_states*int(i), activation=self.conv_activation, kernel_regularizer=self.regularize) for i in 2*2**np.arange(self.conv_layers)]

        self.Pool1   = GlobalMaxPool()
        self.Pool2   = GlobalAvgPool()
        self.Pool3   = GlobalSumPool()

        self.decode  = [Dense(i * hidden_states) for i in  2*2**np.arange(decode_layers+1,1,-1)]
        self.dropout_layers  = [Dropout(dropout) for i in range(len(self.decode))]
        if self.batch_norm:
          self.norm_layers  = [BatchNormalization() for i in range(len(self.decode))]
        else:
          self.norm_layers =  [no_norm for i in range(len(self.decode))]
        
        self.loge     = [Dense(hidden_states) for _ in range(2)]
        self.loge_out = Dense(1)
        self.angles     = [Dense(hidden_states) for _ in range(2)]
        self.angles_out = Dense(2)
        self.angle_scale= Dense(2)
        if n_sigs > 0:
          self.sigs      = [Dense(hidden_states) for i in range(2)]
          self.sigs_out  = Dense(n_sigs)

    def call(self, inputs, training = False):
        x, a, i = inputs
        glob_avg=tf.math.segment_mean(x,i)
        glob_var=abs(tf.math.subtract(tf.math.segment_mean(multiply([x,x]),i),multiply([glob_avg, glob_avg])))
        glob_max=tf.math.segment_max(x,i)
        glob_min=tf.math.segment_min(x,i)
        xglob=tf.concat([glob_avg, glob_var, glob_max, glob_min], axis=1)
        a, e    = self.generate_edge_features(x, a)
        for MP in self.MPs:
          x = MP([x, a, e])
        for conv in self.GCNs:
          x=conv([x,a])
        x1 = self.Pool1([x, i])
        x2 = self.Pool2([x, i])
        x3 = self.Pool3([x, i])
        x = tf.concat([x1, x2, x3], axis = 1)
        x=tf.concat([x, xglob], axis=1)
        for decode_layer, dropout_layer, norm_layer in zip(self.decode, self.dropout_layers, self.norm_layers):
          x = dropout_layer(x, training = training)
          x = self.decode_activation(decode_layer(x))
          x = norm_layer(x, training = training)
                
        x_loge = self.loge[0](x)
        x_loge = self.loge[1](x_loge)
        x_loge = self.loge_out(x_loge)

        x_angles = self.angles[0](x)
        x_angles = self.angles[1](x_angles)
        x_angles = self.angles_out(x_angles)
        zeniazi=sigmoid(self.angle_scale(x_angles))

        if self.n_sigs > 0:
          x_sigs  = self.sigs[0](x)
          x_sigs  = self.sigs[1](x_sigs)
          x_sigs  = tf.abs(self.sigs_out(x_sigs)) + eps
        #could add correlation here 
        xs=tf.stack([x_loge[:,0], zeniazi[:,0]*np.pi, zeniazi[:,1]*2*np.pi], axis = 1)
        if self.n_sigs > 0:
          return tf.concat([xs, x_sigs], axis=1)
        else:
          return xs


    def generate_edge_features(self, x, a):
      send    = a.indices[:, 0]
      receive = a.indices[:, 1]
      
      if self.forward == True:
        forwards  = tf.gather(x[:, 3], send) <= tf.gather(x[:, 3], receive)

        send    = tf.cast(send[forwards], tf.int64)
        receive = tf.cast(receive[forwards], tf.int64)

        a       = SparseTensor(indices = tf.stack([send, receive], axis = 1), values = tf.ones(tf.shape(send), dtype = tf.float32), dense_shape = tf.cast(tf.shape(a), tf.int64))

      diff_x  = tf.subtract(tf.gather(x, receive), tf.gather(x, send))

      dists   = tf.sqrt(
        tf.reduce_sum(
          tf.square(
            diff_x[:, :3]
          ), axis = 1
        ))

      vects = tf.math.divide_no_nan(diff_x[:, :3], tf.expand_dims(dists, axis = -1))

      e = tf.concat([diff_x[:, 3:], tf.expand_dims(dists, -1), vects], axis = 1)

      return a, e

class SGConv(MessagePassing):
    # note that the D^-1/2 norm is not implemented since it is irrelevant for us 
    def __init__(self, n_out, hidden_states, K=2, agg_method='sum', dropout = 0):

        """Agg_method supports "sum": scatter_sum,
          "mean": scatter_mean,
          "max": scatter_max,
          "min": scatter_min,
          "prod": scatter_prod"""
        super().__init__()
        self.n_out = n_out
        self.agg_method=agg_method
        self.K=K
        self.hidden_states = hidden_states
        self.message_mlps = [MLP(hidden_states, hidden = hidden_states * 2, layers = 2, dropout = dropout) for _ in range(self.K)]
        self.update_mlp  = MLP(hidden_states, hidden = hidden_states, layers = 2, dropout = dropout)


    ##inverted structure since tf requires output func to be propagate
    def prop_khop(self, x, a, k, e=None, training = False, **kwargs):
        self.n_nodes = tf.shape(x)[0]
        self.index_i = a.indices[:, 1]
        self.index_j = a.indices[:, 0]

        # Message
        # print(x, a, e)
        # msg_kwargs = self.get_kwargs(x, a, e, self.msg_signature, kwargs)
        messages = self.message(x, a, k, e, training = training)

        # Aggregate
        # agg_kwargs = self.get_kwargs(x, a, e, self.agg_signature, kwargs)

        ##  make own aggregate
        embeddings = self.aggregate(messages, training = training)

        return embeddings

    def propagate(self, x, a, e, training=False):
        for hop in range(self.K):
          x=self.prop_khop(x,a, hop, e, training = training)
        return self.update(x, training = training)

    def message(self, x, a, k, e, training = False):
        # print([self.get_i(x), self.get_j(x), e])
        out = tf.concat([self.get_i(x), self.get_j(x), e], axis = 1)
        out = self.message_mlps[k](out, training = training)
        return out
    
    def update(self, embeddings, training = False):
        out = self.update_mlp(embeddings, training = training)
        return out
    
class MLP(Model):
    def __init__(self, output, hidden=256, layers=2, batch_norm=True,
                 dropout=0.0, activation='relu', final_activation=None):
        super().__init__()
        self.batch_norm = batch_norm
        self.dropout_rate = dropout

        self.mlp = Sequential()
        for i in range(layers):
            # Linear
            self.mlp.add(Dense(hidden if i < layers - 1 else output, activation = activation))
            if dropout > 0:
                self.mlp.add(Dropout(dropout))


    def call(self, inputs, training = False):
        return self.mlp(inputs, training = training)

loading model


In [4]:
model=KHist(hist=1)

In [144]:
batch_size, sig=1024, 0.04
res=40
loader = DisjointLoader(dataset, batch_size=batch_size, epochs=1)
batch=0

In [145]:
@tf.function(input_signature = loader.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs, training = False)
    targets     = tf.cast(targets, tf.float32) 

    return predictions, targets

def predict(loader):
    prediction_list, target_list = [], []
    for batch in loader:
        inputs, targets = batch
        predictions, targets = test_step(inputs, targets)

        prediction_list.append(predictions.numpy())
        target_list.append(targets.numpy())
        y_reco  = tf.concat(prediction_list, axis = 0).numpy()
        y_true  = tf.concat(target_list, axis = 0)
        y_true  = tf.cast(y_true, tf.float32).numpy()
    return y_reco, y_true

In [8]:
inputs, targets = next(loader)
predictions, targets = test_step(inputs, targets)
kdet=KDE_gauss(targets[:,1], sig=sig)
kder=KDE_gauss(predictions[:,1], sig=sig)
if batch==0:
    kdetrue=RunningKDE(kdet)
    kdepred=RunningKDE(kder)
else:
    kdetrue.update(kdet, batch)
    kdepred.update(kder, batch)
batch+=1
tdist, rdist, xs= pdfs(kdetrue.kde, kdepred.kde)
plt.plot(xs, rdist, label='reco')
plt.plot(xs, tdist, label='true')
plt.hist(targets[:,1].numpy(), bins=bins, density=1, alpha=0.4)
plt.hist(predictions[:,1].numpy(), bins=bins, density=1, alpha=0.4);

NameError: name 'loader' is not defined

In [146]:
for i, batch in enumerate(loader):
    inputs, targets = batch
    predictions, targets = test_step(inputs, targets)
    kdet=KDE_gauss(targets[:,1], sig=sig)
    kder=KDE_gauss(predictions[:,1], sig=sig)
    if i==0:
        kdetrue=RunningKDE(kdet)
        kdepred=RunningKDE(kder)
    else:
        kdetrue.update(kdet, i)
        kdepred.update(kder, i)
    tdist, rdist, xs= pdfs(kdetrue.kde, kdepred.kde)
    fig, ax=plt.subplots(figsize=(12,9))
    ax.plot(xs, rdist, label='reco')
    ax.plot(xs, tdist, label='true')
    ax.hist(targets[:,1].numpy(), bins=bins, density=1, alpha=0.4)
    ax.hist(predictions[:,1].numpy(), bins=bins, density=1, alpha=0.4);
    ax.legend()

NameError: name 'KDE_gauss' is not defined

In [ ]:
loader = DisjointLoader(dataset, batch_size=batch_size, epochs=1)
reco, true=predict(loader)


In [ ]:
xs=np.linspace(0,np.pi/2,50)
kder=KDE_gauss(reco[:,1], sig=sig)
kdet=KDE_gauss(true[:,1], sig=sig)
tdist=kdet.prob(xs)
rdist=kder.prob(xs)
plt.plot(xs, rdist, label='reco')
plt.plot(xs, tdist, label='true')
plt.hist(reco[:,1], bins=bins, density=1, alpha=0.4)
plt.hist(true[:,1], bins=bins, density=1, alpha=0.4);

In [ ]:
tdist1, rdist1, xs1= pdfs(kdetrue.kde, kdepred.kde)
fig, ax=plt.subplots(figsize=(12,9))
ax.plot(xs1, tdist1, label='true_loop')
ax.plot(xs, tdist, label='true')
ax.legend()

In [ ]:
mix = 0.3
bimix = tfd.Mixture(
  cat=tfd.Categorical(probs=[mix, 1.-mix]),
  components=[
    kder,
    kdet,
])


In [ ]:
KL=tf.math.reduce_sum(tdist1*tf.math.log((tdist1/(tdist))))
# tfd.kl_divergence(kdet, kder)
KL, tf.keras.losses.KLD(tdist1, tdist)

In [ ]:
plt.plot(xs, bimix.prob(xs))
np.trapz(bimix.prob(xs), xs)

In [ ]:
plt.plot(xs, (yr-yt)**2/np.std(yr-yt)**2)

In [ ]:
np.trapz(np.abs(yr-yt)**2/np.std(yr-yt)**2, xs)

In [5]:
import from_config.dev.eval_model as evals
performance_plot = evals.performance_vM2D

import from_config.dev.metrics as met
metrics=met.energy_angle_zeniazi

import from_config.dev.models as models
reload(models)
model = models.KHop()

import from_config.dev.loss_funcs as lf
loss_func=lf.abs_vonMises2D_angle

loading model
loading model


In [6]:
import os, sys, time, wandb
from tqdm import tqdm
import numpy as np

import os.path as osp

from tensorflow.keras.optimizers import Adam
from spektral.data import DisjointLoader
from importlib import __import__

cwd = osp.abspath('')

# wandblog=True
# batch_size=512
# # Setup Log 
# run = wandb.init(project = 'newloss', entity = "chri862z", group='KDEtest',\
#                  reinit=True)

################################################
#   Load dataset                              #
################################################
from from_config.dev.data_load import graph_data
#load dataset
epochs      = 20
batch_size  = 100

dataset=graph_data(n_data=2e5,transform_path='db_files/muongun/transformers.pkl',\
             db_path= 'db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db', n_neighbors=14, restart=0)

idx_lists = dataset.index_lists
# Split data
dataset_train = dataset[idx_lists[0]]
dataset_val   = dataset[idx_lists[1]]
dataset_test  = dataset[idx_lists[2]]

loader_train = DisjointLoader(dataset_train, epochs=epochs, batch_size=batch_size)
loader_test = DisjointLoader(dataset_test, batch_size=batch_size, epochs=1)


 ###############################################
#   Setup other run params                     #
################################################

early_stop  = 5
patience    = 5
val_epoch = 10

# print('check')
################################################
#   Setup model, loss, lr schedule and metrics #
################################################

# Get model, metrics, lr_schedule and loss function

def classic(lr, warm_up = 3, decay = 0.9):

    def lr_schedule():
        # Intial value
        factor = lr * 1 / 2 ** warm_up
        yield factor
        
        # Multiply with 2 first few round
        for i in range(warm_up):
            factor *= 2
            yield factor

        # Make an exponential decay
        while True:
            factor *= decay
            yield factor

    return lr_schedule

lr_schedule=classic(1e-3)()

# save_path=osp.join(model_path,wandb.run.name)

# if not osp.isdir(save_path):
#     os.makedirs(save_path)
#     print('New folder for saving run made')

# Learning rate and optimizer
learning_rate            = next(lr_schedule)
opt           = Adam(learning_rate)

Loading data to memory


In [13]:
# @tf.function()
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

class RunningKDE():
#     @tf.function(experimental_relax_shapes = True)
    def __init__(self):
        self.k=1
#     @tf.function(experimental_relax_shapes = True)
    def initialize(self):
        self.kde = tfd.Mixture(cat=tfd.Categorical(probs=[tf.cast(0.5, tf.float32), tf.cast(0.5, tf.float32)]),components=[self.kdenew,self.kdenew])
#         self.kde.dtype=tf.float64
#     @tf.function(experimental_relax_shapes = True)
    def update(self, batch):
        mix=1/(batch+1)
        self.kde = tfd.Mixture(cat=tfd.Categorical(probs=[tf.cast(1-mix, tf.float32), tf.cast(mix, tf.float32)]),components=[self.kde,self.kdenew])
#     @tf.function(experimental_relax_shapes = True)
    def prob(self,r=(0,tf.constant(np.pi/2)), res=50):
        xs=tf.linspace(tf.cast(r[0], dtype=tf.float32),tf.cast(r[1], dtype=tf.float32),res)
        return self.kde.prob(xs)
#     @tf.function(experimental_relax_shapes = True)
    def KDE_gauss(self, x,  batch_size=100, sig=0.05):
#         @tf.function(experimental_relax_shapes = True)
        def f(x, sig):
            tfd.Independent(tfd.Normal(
            loc=x, scale=sig))
        n = batch_size
        self.kdenew = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(
        probs=[1/n]*n),
        components_distribution=f(x, sig))
        
#     @tf.function()
#     def prob(self):
#         self.kde.prob(1)
#         return tf.constant([value])
# pi=np.pi
# def KLD(kdet, kder, r=(0,tf.constant(np.pi)/2), res=50):
#     xs=tf.linspace(tf.cast(r[0], dtype=tf.float32),tf.cast(r[1], dtype=tf.float32),res)
#     tdist=kdet.prob(xs)
#     rdist=kder.prob(xs)
#     return tf.math.reduce_sum(tdist*tf.math.log(tdist/rdist))

# def pdfs(true, reco, r=(0,tf.constant(np.pi/2)), res=50):
#     xs=tf.linspace(tf.cast(r[0], dtype=tf.float64),tf.cast(r[1], dtype=tf.float64),res, dtype=tf.float64)
#     tdist=true.prob(xs)
#     rdist=reco.prob(xs)
#     return tdist, rdist, xs

In [14]:
# @tf.function(experimental_relax_shapes = True )
def loss_kld(reco, true, kdetrue, kdepred, i=0):
#     tf.print(true, reco)
    kdetrue.KDE_gauss(true[:,1])
    kdepred.KDE_gauss(reco[:,1])
    if i==0:
        kdetrue.initialize()
        kdepred.initialize()
#         tf.print(i)
    else:
        kdetrue.update(i)
        kdepred.update(i)
    tf.print(kdetrue.kde._cat_probs(log_probs=1))
#     tdist=kdetrue.prob()
#     rdist=kdepred.prob(4)
#     tdist, rdist=tf.Tensor([1], dtype=tf.float32), tf.Tensor([0], dtype=tf.float32)
#     tdist, rdist=tf.constant([1], dtype=tf.float32), tf.constant([0.1], dtype=tf.float32)
#     return tf.keras.losses.KLD(tdist, rdist)
    return 1

In [15]:
# @tf.function(experimental_relax_shapes = True )
# def loss_kld(reco, true):
#     return tf.keras.losses.KLD(true[:,1], reco[:,1])
# #     return 1

In [16]:
class keep_track():
#         @tf.function(experimental_relax_shapes = True)
        def __init__(self):
            self.list=[]
        def update(self, l):
            self.list=tf.concat([self.list, l], axis=0)

In [18]:
wandblog=0

################################################
#   Set up TF functions and validation step   #
################################################

# Define training function
# @tf.function(input_signature = loader_train.tf_signature(), experimental_relax_shapes = True)
TensorSpec=tf.TensorSpec
TensorShape=tf.TensorShape
SparseTensorSpec=tf.SparseTensorSpec
new_sig=(((TensorSpec(shape=(None, 6), dtype=tf.float64, name=None),
   SparseTensorSpec(TensorShape([None, None]), tf.float64),
   TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
  TensorSpec(shape=(None, 3), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None)))
keep=keep_track()
# @tf.function(input_signature = new_sig, experimental_relax_shapes = True)
# @tf.function(experimental_relax_shapes = True)
def train_step(inputs, targets, kdetrue, kdepred, i):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        targets     = tf.cast(targets, tf.float32)
        loss        = loss_func(predictions, targets)
#         keep.update(targets[:,1])
#         loss       +=loss_kld(predictions, targets)
        loss1       =loss_kld(predictions, targets, kdetrue,kdepred, i)
        kdetrue.kde.prob(1)
        loss       += sum(model.losses)
#         tf.print(i)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# @tf.function(input_signature = loader_test.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs, training = False)
    targets     = tf.cast(targets, tf.float32) 

    return predictions, targets, out


def validation(loader):
    loss = 0
    prediction_list, target_list = [], []
    for batch in loader:
        inputs, targets = batch
        predictions, targets, out = test_step(inputs, targets)
        loss           += out

        prediction_list.append(predictions)
        target_list.append(targets)

    y_reco  = tf.concat(prediction_list, axis = 0)
    y_true  = tf.concat(target_list, axis = 0)
    y_true  = tf.cast(y_true, tf.float32)

    loss, loss_from = loss_func(y_reco, y_true, re=True)

    energy, e_old, alpha, zeni, azi= metrics(y_reco, y_true)

    return loss, loss_from, [energy, e_old, alpha, zeni, azi]



################################################
#  Train Model                                 #      
################################################

kdetrue=RunningKDE()
kdepred=RunningKDE()

tot_time=0
current_batch = 0
current_epoch = 1
loss          = 0
lowest_loss   = np.inf
early_stop    = 1
early_stop_counter    = 0
pbar          = tqdm(total = loader_train.steps_per_epoch, position=0, leave = True)
start_time    = time.time()
summarylist=[]
for i, batch in enumerate(loader_train):
    i=tf.cast(i, tf.int32)
    
    inputs, targets = batch
    out             = train_step(inputs, targets, kdetrue, kdepred, i=i)
    loss           += out
    if current_epoch==1 and current_batch==0:
        model.summary()
        if wandblog:
            summary=model.summary(print_fn=summarylist.append)
            table=wandb.Table(columns=["Layers"])
            for s in summarylist:
                table.add_data(s)
            wandb.log({'Model summary': table})
    current_batch  += 1
    pbar.update(1)
    pbar.set_description(f"Epoch {current_epoch} / {epochs}; Avg_loss: {loss / current_batch:.6f}")

    if current_batch == loader_train.steps_per_epoch:
        t=time.time() - start_time
        tot_time+=t
        print(f"Epoch {current_epoch} of {epochs} done in {t:.2f} seconds using learning rate: {learning_rate:.2E}")
        print(f"Avg loss of train: {loss / loader_train.steps_per_epoch:.6f}")

        loader_val    = DisjointLoader(dataset_val, epochs = 1,      batch_size = batch_size)
        val_loss, val_loss_from, val_metric = validation(loader_val)
        if wandblog:
            wandb.log({"Train Loss":      loss / loader_train.steps_per_epoch,
                    "Validation Loss": val_loss, 
                    "w(log(E))":   val_metric[1],
                    "Energy bias":   val_metric[0][1],
                    "Energy sig-1":   val_metric[0][0],
                    "Energy sig+1":   val_metric[0][2],
                    "Solid angle 68th":    val_metric[2][3],
                    "Angle bias":   val_metric[2][1],
                    "Angle sig-1":   val_metric[2][0],
                    "Angle sig+1":   val_metric[2][2],
                    "zenith 68th":    val_metric[3][3],
                    "zenith bias":   val_metric[3][1],
                    "zenith sig-1":   val_metric[3][0],
                    "zenith sig+1":   val_metric[3][2],
                    "azimuth 68th":    val_metric[4][3],
                    "azimuth bias":   val_metric[4][1],
                    "azimuth sig-1":   val_metric[4][0],
                    "azimuth sig+1":   val_metric[4][2],
                    "Learning rate":   learning_rate})
        print("\n")
        if not construct_dict['run_params']['zeniazi_metric']:
            print(f"Avg loss of validation: {val_loss:.6f}")
            print(f"Loss from:  Energy: {val_loss_from[0]:.6f} \t Angle: {val_loss_from[1]:.6f} ")
            print(f"Energy: bias = {val_metric[0][1]:.6f} sig_range = {val_metric[0][0]:.6f}<->{val_metric[0][2]:.6f}, old metric {val_metric[1]:.6f}\
                \n Angle: bias = {val_metric[2][1]:.6f} sig_range = {val_metric[2][0]:.6f}<->{val_metric[2][2]:.6f}, old metric {val_metric[2][3]:.6f}")
        else:
            print(f"Avg loss of validation: {val_loss:.6f}")
            print(f"Loss from:  Energy: {val_loss_from[0]:.6f} \t Angle: {val_loss_from[1]:.6f} ")
            print(f"Energy: bias = {val_metric[0][1]:.6f} sig_range = {val_metric[0][0]:.6f}<->{val_metric[0][2]:.6f}, old metric {val_metric[1]:.6f}\
                \n Angle: bias = {val_metric[2][1]:.6f} sig_range = {val_metric[2][0]:.6f}<->{val_metric[2][2]:.6f}, old metric {val_metric[2][3]:.6f}\
                \n Zenith: bias = {val_metric[3][1]:.6f} sig_range = {val_metric[3][0]:.6f}<->{val_metric[3][2]:.6f}, old metric {val_metric[3][3]:.6f}\
                \n Azimuth: bias = {val_metric[4][1]:.6f} sig_range = {val_metric[4][0]:.6f}<->{val_metric[4][2]:.6f}, old metric {val_metric[4][3]:.6f}")

        if val_loss < lowest_loss:
            early_stop_counter = 0
            lowest_loss        = val_loss
        else:
            early_stop_counter += 1
        print(f'Early stop counter: {early_stop_counter}/{patience}, lowest val loss was {lowest_loss:.6f}')
        if early_stop and (early_stop_counter >= patience):
            print(f"Stopped training. No improvement was seen in {patience} epochs")

        if current_epoch != epochs:
            pbar          = tqdm(total = loader_train.steps_per_epoch, position=0, leave = True)

        learning_rate = next(lr_schedule)
        opt.learning_rate.assign(learning_rate)

        time_avg=tot_time/current_epoch
        if current_epoch % val_epoch == 0:
            model.save(save_path)
            print("Model saved")
            if wandblog:
                loader_test = DisjointLoader(dataset_test, batch_size=batch_size, epochs=1)
                fig, _ = performance_plot(loader_test, test_step, metrics, save=True, save_path=save_path)
                title="performanceplot_"+str(current_epoch)
                wandb.log({title: [wandb.Image(fig, caption=title)]})

        loss            = 0
        start_time      = time.time()
        current_epoch  += 1
        current_batch   = 0


  0%|                                                                                         | 0/1600 [00:00<?, ?it/s]

RuntimeError: in user code:

    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\spektral\layers\convolutional\message_passing.py:100 call  *
        return self.propagate(x, a, e)
    C:\Users\chris\Christian\3YR-UNI\Bachelor\IceCube\from_config\dev\models.py:490 propagate  *
        x=self.prop_khop(x,a, hop, e, training = training)
    C:\Users\chris\Christian\3YR-UNI\Bachelor\IceCube\from_config\dev\models.py:478 prop_khop  *
        messages = self.message(x, a, k, e, training = training)
    C:\Users\chris\Christian\3YR-UNI\Bachelor\IceCube\from_config\dev\models.py:496 message  *
        out = self.message_mlps[k](out, training = training)
    C:\Users\chris\Christian\3YR-UNI\Bachelor\IceCube\from_config\dev\models.py:332 call  *
        return self.mlp(inputs, training = training)
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:786 __call__  **
        outputs = call_fn(cast_inputs, *args, **kwargs)
 

In [ ]:
# kdetrue.kde.prob(1)